# 3. Ocean forcing

*Author: Andy Aschwanden*

In this tutorial, we will visualize the ocean forcing provided by [Ice Sheet Model Intercomparison for CMIP6 (ISMIP6)](https://climate-cryosphere.org/about-ismip6/]).

In [ ]:
from functools import partial

from dask.diagnostics import ProgressBar
from pathlib import Path
import matplotlib.pylab as plt
from matplotlib import colors
import numpy as np
import xarray as xr
import cf_xarray.units  # otherwise we get a Parser error
import pint_xarray
import seaborn as sns

from pism_tutorials.processing import preprocess_nc
from pism_tutorials.utils import merge_dicts, dict2str, sort_dict_by_key

xr.set_options(keep_attrs=True)                   

## Sync with pism-cloud-data

We need data to serve as initial and boundary conditions as well as observations for validation. Data pre-processing can be a large part of project and the time it takes to get ready to perform a simulation is easily underestimated. For the purposes of this tutorial, we have done the hard part already and put all necessary data into a public S3 bucket.

Because the python library `boto3` lacks a command to easily *synchronize a local directory with an *s3* bucket, we use the *aws command line tools* (`awscli`) that should have been installed when you generated the `conda` environment, if not, you can run `pip install awscli`. We are downloading a few `GB`, so the intial sync might take a while.


In [ ]:
# The name of the PISM Cloud S3 bucket
bucket_name = "pism-cloud-data"
! aws s3 sync --no-sign-request s3://pism-cloud-data/tutorial_files  tutorial_files
! aws s3 sync --no-sign-request s3://pism-cloud-data/ismip6_ocean  ismip6_ocean
! aws s3 sync --no-sign-request s3://pism-cloud-data/glacier_dems  glacier_dems

## ISMIP6 Ocean Forcing



In [ ]:
ds = xr.open_mfdataset("ismip6_ocean/MAR3.9_*_theta_ocean_1980-2020_v4.nc", 
                       preprocess=partial(preprocess_nc, regexp="9_(.+?)_theta_ocean", dim="GCM"), 
                       parallel=True,             
                       combine="nested",
                       concat_dim="GCM",
                       chunks="auto",)

### Petermann Gletscher

We now subset a point on the shelf and load in the DEM for the area.

In [ ]:
petermann_loc = {"x": -275140, "y": -949491}
petermann_gp = gp.GeoSeries(Point(petermann_loc.values()), crs="EPSG:3413")
petermann_dem = xr.open_dataset("glacier_dems/petermann.nc")
petermann_theta = ds.sel(petermann_loc, method="nearest").theta_ocean.compute()

Plot the position of the point on the map of subglacial topography.

In [ ]:
fig, ax = plt.subplots(1, 1)
petermann_dem.bed.plot(ax=ax)
petermann_gp.plot(ax=ax, color="k")


The time series is of `theta_ocean` at the point is:

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(10.2, 3.6))
petermann.plot(hue="GCM", ax=ax, add_legend=True)
ax.plot(xr.date_range("2000", "2005", freq="YS"),[2.6] * 6)
sns.move_legend(plt.gca(), loc='center left', bbox_to_anchor=(1, 0.5))
ax.set_title("Petermann Gletscher")
plt.tight_layout()
plt.show()

Now we can set up PISM.

In [ ]:
resolution = "450m"  # a coarse resolution such that we can run the simulation on a desktop computer

output_dir = Path("basal_melt_rates")
output_dir.mkdir(parents=True, exist_ok=True)

extra_file = ""
start = "2000-01-01"
end = "2001-01-01"


input_params = {
    "bootstrap": "",
    "i": None,
}

grid_params = {
    "grid.dx": resolution,
    "grid.dy": resolution,
    "grid.Mz": 101,
    "grid.Lz": 4000,
    "grid.Mbz": 11,
    "grid.Lbz": 1000,
}

time_params = {
    "time.start": start,
    "time.end": end,
    "time.calendar": "standard",
    "time_stepping.skip.enabled": "",
    "time_stepping.skip.max": 100
}

climate_params = {
    "surface": "elevation",
    "ocean.models": "th",
    "ocean.th.file": None,
    "constants.sea_water.salinity": 35,
    "ocean.th.gamma_T": 0.0001,
    "ocean.th.clip_salinity": "no"
}

dynamics_params = {
    "no_mass": "",
    "energy": "none",
    "stress_balance.sia.max_diffusivity": 1000000,
 
}

output_params = {
    "output.extra.file": extra_file,
    "output.extra.times": "yearly",
    "output.extra.vars": "bmelt,mask,topg,usurf,thk",
    "output.size": "none"  # do not write a state file
}

run_dict = merge_dicts(input_params, grid_params, time_params, dynamics_params, climate_params, output_params)

In [ ]:
for glacier in ["petermann", "jib", "79n"]:
    spatial_files = output_dir.glob(f"*{glacier}*.nc")
    spatial_ds = xr.open_mfdataset(
        spatial_files,
        preprocess=partial(preprocess_nc, regexp="gcm_(.+?)_", dim="GCM"), 
                       parallel=True,             
                       combine="nested",
                       concat_dim="GCM",
                       chunks="auto",)
    spatial_ds = spatial_ds.where(spatial_ds)
    f = spatial_ds["bmelt"].plot(col="GCM", col_wrap=3)